In [ ]:
!rm gaussian-splatting -rf
!git clone --recursive https://github.com/graphdeco-inria/gaussian-splatting

In [ ]:
%pip install xformers==0.0.19 torch==2.0.0+cu118 torchvision  torchaudio --extra-index-url https://download.pytorch.org/whl/cu118

In [ ]:
%pip install plyfile==0.8.1 tqdm


In [ ]:
%cd /notebooks/gaussian-splatting
%cd ./submodules/diff-gaussian-rasterization
%pip install .
%cd /notebooks/gaussian-splatting

In [ ]:
%cd /notebooks/gaussian-splatting
%cd ./submodules/simple-knn
%pip install .
%cd /notebooks/gaussian-splatting

In [ ]:
%cd /notebooks/gaussian-splatting
!git pull origin main

In [ ]:
!sudo apt update
!sudo apt install -y libglew-dev libassimp-dev libboost-all-dev libgtk-3-dev libopencv-dev libglfw3-dev libavdevice-dev libavcodec-dev libeigen3-dev libxxf86vm-dev libembree-dev


In [ ]:
!git clone https://github.com/ImageMagick/ImageMagick.git ImageMagick-7.1.1
%cd ImageMagick-7.1.1
!./configure
!make
!make install

In [ ]:
!ldconfig /usr/local/lib
!magick -version

TRAIN

In [ ]:
%pip install -q plyfile
%pip install -q https://huggingface.co/camenduru/gaussian-splatting/resolve/main/diff_gaussian_rasterization-0.0.0-cp310-cp310-linux_x86_64.whl
%pip install -q https://huggingface.co/camenduru/gaussian-splatting/resolve/main/simple_knn-0.0.0-cp310-cp310-linux_x86_64.whl



In [ ]:

!wget https://huggingface.co/camenduru/gaussian-splatting/resolve/main/tandt_db.zip
!unzip tandt_db.zip


In [ ]:
%pip install gdown


In [ ]:
!rm -rf  /notebooks/test

In [ ]:
!mkdir /notebooks/xxx

In [ ]:
!unzip -d /notebooks/xxx/input "/notebooks/Archive.zip" -x "__MACOSX/*"

In [ ]:
import os

directory = '/notebooks/test/data/keyframes/images/'  # specify your directory

# Get all files from the directory
files = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]

# Filter out non-jpeg files
jpeg_files = [f for f in files if f.endswith('.jpeg')]

# Sort files (if necessary)
jpeg_files.sort()

# Rename files
for i, filename in enumerate(jpeg_files):
    old_path = os.path.join(directory, filename)
    new_path = os.path.join(directory, f'{i:05d}.jpeg')  # new name will be 00000.jpeg, 00001.jpeg, etc.
    os.rename(old_path, new_path)

In [ ]:
directory = '/notebooks/test/data/keyframes/images/'

# Get all files from the directory
files = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]

# Filter out non-jpeg files
jpeg_files = [f for f in files if f.endswith('.jpeg') or f.endswith('.jpg')]  # Also consider '.jpg' extension

# Sort files (if necessary)
jpeg_files.sort()

# Convert JPEG files to PNG using ImageMagick's magick command
for jpeg_file in jpeg_files:
    source = os.path.join(directory, jpeg_file)
    destination = os.path.splitext(source)[0] + '.png'
    !magick "$source" "$destination"


In [ ]:
%cd /notebooks/gaussian-splatting
%run convert.py -s /notebooks/xxx


In [ ]:
!mkdir /notebooks/xxx/images

In [ ]:
%cp /notebooks/xxx/input/* /notebooks/xxx/images

In [ ]:
from sys import exit
%run convert.py -s  /notebooks/test/data/keyframes  --skip_matching --resize

In [ ]:
%cd /notebooks/gaussian-splatting
%run train.py  -s /notebooks/xxx

In [ ]:
%cd /notebooks/gaussian-splatting
%run  render.py -m  /notebooks/gaussian-splatting/output/0fbca5a0-3 -s /notebooks/tablexxx2

In [ ]:
%run  metrics.py -m  /notebooks/gaussian-splatting/output/193d95f0-5 

In [ ]:
!zip -r 193d95f0-5.zip /notebooks/gaussian-splatting/output/193d95f0-5 

In [ ]:
!zip -r test-2.zip /notebooks/test-2

In [ ]:
%cd /notebooks/gaussian-splatting/output/930e0cf7-f 

In [ ]:
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb -O cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb
%pip install nest-asyncio

In [ ]:
%autoawait True
import nest_asyncio
nest_asyncio.apply()
import atexit, requests, subprocess, time, re, os
from random import randint
from threading import Thread
from queue import Queue

def cloudflared(port, metrics_port, output_queue):
    atexit.register(lambda p: p.terminate(), subprocess.Popen(['cloudflared', 'tunnel', '--url', f'http://127.0.0.1:{port}', '--metrics', f'127.0.0.1:{metrics_port}'], stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT))
    attempts, tunnel_url = 0, None
    while attempts < 10 and not tunnel_url:
        attempts += 1
        time.sleep(3)
        try:
            tunnel_url = re.search("(?P<url>https?:\/\/[^\s]+.trycloudflare.com)", requests.get(f'http://127.0.0.1:{metrics_port}/metrics').text).group("url")
        except:
            pass
    if not tunnel_url:
        raise Exception("Can't connect to Cloudflare Edge")
    output_queue.put(tunnel_url)

output_queue, metrics_port = Queue(), randint(8100, 9000)
thread = Thread(target=cloudflared, args=(7860, metrics_port, output_queue))
thread.start()
thread.join()
tunnel_url = output_queue.get()
os.environ['webui_url'] = tunnel_url
print(tunnel_url)


%run -m http.server 7860

COLMAP

In [ ]:
!sudo apt-get -y install \
    git \
    cmake \
    ninja-build \
    build-essential \
    libboost-program-options-dev \
    libboost-filesystem-dev \
    libboost-graph-dev \
    libboost-system-dev \
    libeigen3-dev \
    libflann-dev \
    libfreeimage-dev \
    libmetis-dev \
    libgoogle-glog-dev \
    libgtest-dev \
    libsqlite3-dev \
    libglew-dev \
    qtbase5-dev \
    libqt5opengl5-dev \
    libcgal-dev \
    libceres-dev


In [ ]:
!git clone https://github.com/colmap/colmap.git
%cd colmap
!git checkout dev
!mkdir build
%cd build
!cmake .. -GNinja  -DCMAKE_CUDA_ARCHITECTURES=native
!ninja
!ninja install